### Functional API简介

In [1]:
import os
import numpy as np

In [2]:
from keras import Input, layers

# Input tensor
input_tensor = Input(shape=(32,))

# A layer is a function
dense = layers.Dense(32, activation='relu')

# A layer may be called on a tensor, and it returns a tensor
output_tensor = dense(input_tensor)

Using TensorFlow backend.


In [3]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

In [4]:
# Sequential model
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))
seq_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Functional model
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

# The Model class turns an input tensor and output tensor into a model
model = Model(input_tensor, output_tensor)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


对这种`Model`实例进行compile、train和evaluate时，其API与`Sequential`模型相同。

In [6]:
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit(x_train, y_train, batch_size=138, epochs=10, verbose=0)
score = model.evaluate(x_train, y_train)

1000/1000 [==============================] - 0s 49us/step


In [7]:
score

11.503647827148438

**使用函数式模型的一个典型场景是搭建多输入、多输出的模型。**

### 多输入模型

通常情况下，模型会在某一时刻用一个可以组合多个张量的层将不同的输入分支合并，张量组合方式可能是相加（`keras.layers.add`）、连接（`keras.layers.concatenate`）等。

一个简单**问答模型**：一个自然语言描述的问题和一个文本片段（比如新闻），后者提供用于回答问题的信息。然后模型要生成一个回答，在最简单的情况下，这个回答只包含一个词，可以通过对某个预定义的词表做`softmax`得到。

In [8]:
from keras.models import Model
from keras import layers, Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# 文本输入是一个长度可变的整数序列，可以选择对输入进行命名
text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

# 对文本进行相同的处理
question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(
    question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# 将编码后的问题和文本连接起来
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

# 在上面添加一个`softmax`分类器
answer = layers.Dense(
    answer_vocabulary_size, activation='softmax')(concatenated)

# Model实例化时，指定两个输入和一个输出
model = Model([text_input, question_input], answer)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [9]:
 model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 32)     320000      question[0][0]                   
__________________________________________________________________________________________________
lstm_1 (LS

**训练双输入模型，有两个可用的API：**
1. 可以向模型输入一个有Numpy数组组成的列表；
2. 可以输入一个将输入名称映射为Numpy数组的字典，只有输入具有名称时才能使用这种方法。

In [10]:
from keras.utils import to_categorical

In [11]:
num_samples = 1000
max_len = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_len))
question = np.random.randint(1, question_vocabulary_size,
                             size=(num_samples, max_len))
answers = np.random.randint(answer_vocabulary_size, size=(num_samples))
# 回答是one-hot编码，不是整数
answers = to_categorical(answers, answer_vocabulary_size)

```python
# 第一种训练方式
model.fit([text, question], answers, epochs=10, batch_size=128, verbose=1)

# 第二种训练方式
model.fit({'text': text, 'question': question}, answers,
          epochs=10, batch_size=128)
```

### 多输出模型

一个简单的例子是一个网络试图同时预测数据的不同性质，比如一个网络，输入某个匿名人士的一系列社交媒体发帖，然后尝试预测那个人的属性，如年龄、性别和收入水平。

In [12]:
# Functional API implementation of a three-output model
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# 输出层都具有名称
age_pred = layers.Dense(1, name='age')(x)
income_pred = layers.Dense(
    num_income_groups, activation='softmax', name='income')(x)
gender_pred = layers.Dense(1, activation='sigmoid', name='gender')(x)
model = Model(posts_input, [age_pred, income_pred, gender_pred])

In [13]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 50000)  12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 128)    32000128    embedding_3[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, None, 128)    0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (

**训练这种模型需要能够对网络的各个输出制定不同的损失函数。**年龄预测是标量回归任务，而性别预测是二分类任务，二者需要不同的训练过程。但是梯度下降要求将一个标量最小化，所以为了能够训练模型，必须将这些损失合并为单个标量。合并不同损失最简单的方法就是对所有损失求和。

**在Keras中，可以在compile时使用损失组成的列表或字典来为不同输出制定不同损失，然后将得到的损失值相加得到一个全局损失，并在训练过程中将这个损失最小化。**

In [14]:
# 多输出模型的编译选项：多重损失
# 第一种方式
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

In [15]:
# 第二种方式，只有输出层具有名称时才能采用这种方式
model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'})

严重不平衡的损失会导致模型表示针对单个损失最大的任务优先进行优化，而不考虑其他任务的优化。为解决这个问题，可以为每个损失值对最终损失的贡献分配不同大小的重要性。如果不同的损失值具有不同的取值范围，那么这一方法尤其有用。

In [16]:
# 多输出模型的编译选项：损失加权
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy','binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])

In [17]:
model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25,
                            'income': 1.,
                            'gender': 10.})

与多输入模型相同，多输出模型的训练输入数据可以是Numpy数组组成的列表或字典。

```python
# 第一种训练方式
model.fit(posts, [age_targets, income_targets, gender_targets],
          epochs=10, batch_size=64)

# 第二种训练方式
model.fit(posts, {'age': age_targets,
                  'income': income_targets,
                  'gender': gender_targets},
          epochs=10,
          batch_size=64)
```

### 层组成的有向无环图

1. Inception模块
2. 残差连接

### 共享层权重

函数式API还有一个重要特性，那就是能够多次重复使用一个层实例。

连体LSTM或共享LSTM模型：

```python
from keras import layers
from keras import Input
from keras.models import Model

lstm = layers.LSTM(32)   # 将一个LSTM层实例化

# 构建模型的左分支：输入是长度128的向量组成的变长序列
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

# 构建模型的右分支：如果调用已有的层实例，那么就会重复使用它的权重
right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1)

# 在上面构建一个分类器
predictions = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions)

# 训练这种模型时，基于两个输入对LSTM层的权重进行更新
model.fit([left_input, right_input], targets)
```

### 将模型作为层

可以在一个输入张量上调用模型，并得到一个输出张量：

```python
y = model(x)
```

如果模型具有多个输入和多个输出张量，那么应该用张量列表来调用模型：

```python
y1, y2 = model([x1, x2])
```

**在调用模型是实例时，就是在重复使用模型的权重，正如在调用层实例时，就是在是使用层的权重。无论哪一种，都是调用这个实例已经学到的表示。**

使用双摄像头作为输入的视觉模型：两个平行的摄像头，相距几厘米。这样的模型可以感知深度，很有用。不需要两个单独的模型从左右两个摄像头中分别提取视觉特征，然后再将二者合并。这样的底层处理可以在两个输入之间共享，即通过共享层（使用相同的权重，从而共享相同的表示）来实现。

实现连体视觉模型（共享卷积层）：

In [18]:
from keras import Input
from keras import layers
from keras.applications import Xception

# 图像处理基础模型是Xception网络（只包含卷积基）
xception_base = Xception(weights=None, include_top=False)

left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

left_features = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_features], axis=-1)